In [1]:
''' 1. 모듈 임포트 '''
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim
from torchvision import transforms, datasets

In [2]:
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.5.0+cu101  Device: cuda


In [3]:
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
''' 3. FashionMNIST 데이터 다운로드 '''
trainset = datasets.FashionMNIST(
    root      = '../data/FashionMNIST', 
    train     = True,
    download  = True,
    transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True)

KeyboardInterrupt: 

In [ ]:
''' 4. 데이터 확인하기 (1) '''
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break 

In [ ]:
''' 4. 데이터 확인하기 (2) '''
pltsize = 1
plt.figure(figsize=(10 * pltsize, pltsize))

for i in range(10):
    plt.subplot(1, 10, i + 1)
    plt.axis('off')
    plt.imshow(X_train[i, :, :, :].numpy().reshape(28, 28), cmap = "gray_r")
    plt.title('Class: ' + str(y_train[i].item()))

In [ ]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),)

        self.decoder = nn.Sequential(
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 28 * 28),
            nn.Sigmoid())

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

model = AE().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr = 0.005)
criterion = nn.MSELoss()

print("Model: ", model)
print("Device: ", DEVICE)

In [ ]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.view(-1, 28 * 28).to(DEVICE)
        target = image.view(-1, 28 * 28).to(DEVICE)

        optimizer.zero_grad()
        encoded, decoded = model(image)
        loss = criterion(decoded, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
def get_lv(model, train_loader):
    lv_lst = []
    model.eval()
    with torch.no_grad():
        for image, _ in train_loader:
            image = image.view(-1, 28 * 28).to(DEVICE)
            encoded, _ = model(image)
            lv_lst.append(encoded.to("cpu").numpy())
    lv = np.reshape(np.array(lv_lst), (-1, 16))
    return lv

In [ ]:
''' Training'''
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    print("\n")

In [ ]:
lv = get_lv(model, train_loader)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators = 20,
                                  max_depth = 10,
                                  random_state = 42)
rf_model_trained = rf_model.fit(trainset.train_data.view(-1, 28 * 28).numpy(), trainset.train_labels.numpy())
rf_model_predicted_train = rf_model.predict(trainset.train_data.view(-1, 28 * 28).numpy())
print("Trainset Accuracy: %.2f" % (accuracy_score(list(trainset.train_labels.numpy()), rf_model_predicted_train) * 100), "%") 

In [ ]:
rf_model = RandomForestClassifier(n_estimators = 20,
                                  max_depth = 10,
                                  random_state = 42)
rf_model_trained = rf_model.fit(lv, trainset.train_labels.numpy())
rf_model_predicted_train = rf_model.predict(lv)
print("Trainset Accuracy: %.2f" % (accuracy_score(list(trainset.train_labels.numpy()), rf_model_predicted_train) * 100), "%") 

In [ ]:
lv.shape

In [ ]:
trainset.train_data.numpy().shape